In [ ]:
%load_ext autoreload
%autoreload 2

from F5522A import F5522A
import traceback
import time

print ("Start")

try:
    Calibrator = F5522A('COM6')
    
    print(Calibrator.read(b'ERR?'))
    
    print(Calibrator.read(b'ERR?'))
    
    Calibrator.Operate()
    Calibrator.write(b'OUT 65V\n')
    
    time.sleep(10)
    
    Calibrator.disconnect()
    
    print ("Done")

except Exception as e:
    #exception handling code
    print(traceback.format_exc())
    Calibrator.disconnect()

In [ ]:
import visa
from visa import constants
import time

rm = visa.ResourceManager()
insts = rm.list_resources()

inst = rm.open_resource("ASRL6::INSTR")

inst.flow_control=visa.constants.VI_ASRL_FLOW_XON_XOFF

inst.timeout = 10000
print (inst.read_termination)

inst.baudrate = 9600
inst.data_bits = 8
print (inst.data_bits)
print (inst.stop_bits)
print (inst.parity)
print (inst.baud_rate)
inst.write_termination = '\r\n'


inst.open()

inst.query_ascii_values('*IDN?')


inst.close()


In [ ]:
import visa
from visa import constants
import time

rm = visa.ResourceManager()
insts = rm.list_resources()



print (insts)

In [ ]:
%load_ext autoreload
%autoreload 2

from F5522A import F5522A
from F5790 import F5790
from K53220A import K53220A
import traceback
import time
import visa

from master import TestPoint,Range ,slackmessenger

TP = TestPoint('1.9000 mA~@ 2.1 MHz')

print (TP.toList())

Meter = K53220A()

#Meter.SetRange(TP,'Sour Bus')
Meter.SetRange(TP,'Frequency Read')


print (Meter.TakeReading())

Meter.disconnect()

# print (Meter.Identity())

# #print (Meter.CustomQuery('READ?'))

# print (Meter.TakeReading())

# Meter.disconnect()



In [ ]:
print ("hello")

In [ ]:
import pandas as pd

Ranges={}

df_ranges=pd.read_excel('Ranges.xlsx',sheet_name='Keysight 53220A')

for index,row in df_ranges.iterrows():

    Ranges[row['Function Name']]=Range(row['Function Name'],row['Primary Unit'],row['Primary Limit Low'],
                             row['Primary Limit High'],row['Secondary Unit'],row['Secondary Limit Low'],
                             row['Secondary Limit High'],row['Command'])

Splitter=Ranges['Phase'].FunctionCommand

for n in Splitter.split('|'):
    print (n.encode())

In [31]:
%load_ext autoreload
%autoreload 2

from F5522A import F5522A
from HP3458 import HP3458
from F5730A import F5730A
import traceback
import time
import pandas as pd
import numpy as np

from master import TestPoint,Range ,slackmessenger

try:
    Meter = HP3458(22)
    Calibrator = F5522A('COM6')
    Standard = F5730A('COM3')
    
    TP_UUT = TestPoint('2.0000 mF')
    TP_ST = TestPoint('540 µA')
    
    Calibrator.SetRange(TP_UUT,'Capacitance')
    Standard.SetRange(TP_ST,'DC Current')
    
    TP = TestPoint('10.00000 V')
    Meter.SetRange(TP,'DCV 10 V')
    
    Calibrator.Operate()
    Standard.Operate()
    
    data = Meter.TakeReading()
    
    Calibrator.Standby()
    Standard.Standby()
    
except Exception as e:
    print (e)
    Meter.disconnect()
    Calibrator.disconnect()
    Standard.disconnect()
    
    print ("Done")
    
Meter.disconnect()
Calibrator.disconnect()
Standard.disconnect()


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
b'REMOTE\n'
b'*CLS\n'
b'STBY\n'
b'ISR?\n'
Output Settled
b'OUT 0.002 F\n'
Output Settled
['OUT 0.00054 A, 0 HZ']
DCV 10; APER 1.0e-3; MEM FIFO; SWEEP 0.1, 100; END ALWAYS
b'ISR?\n'
0
b'*CLS\n'
b'OPER\n'
b'ISR?\n'
b'ISR?\n'
b'ISR?\n'
b'ISR?\n'
Output Settled
0
Output Settled
b'STBY\n'
b'ISR?\n'
b'ISR?\n'
b'ISR?\n'
Output Settled
Output Settled
b'LOCAL\n'
b'STBY\n'
b'ISR?\n'
Output Settled


In [32]:
def parse(reading):
    
    reading = reading['Readings']

    reading=float(reading.split('E')[0].strip('+'))*10**int(reading.split('E')[1])
    
    return reading

df = pd.DataFrame(np.array(data), columns = ['Readings'])
df['Readings']=df['Readings'].str.replace('\r\n','')
df['Readings']=df.apply(parse,axis=1)

delta_V=df.iloc[0]['Readings']-df.iloc[-1]['Readings']

Current = TP_ST.ComputeValue('Primary')


result = (Current * 9.9) / delta_V


print (delta_V,Current,result)


2.6726095681 0.00054 0.0020002921727922105


In [33]:
print (df)

    Readings
0   3.448656
1   3.421676
2   3.394685
3   3.367687
4   3.340686
5   3.313694
6   3.286700
7   3.259699
8   3.232722
9   3.205730
10  3.178735
11  3.151747
12  3.124755
13  3.097760
14  3.070754
15  3.043756
16  3.016773
17  2.989749
18  2.962783
19  2.935777
20  2.908782
21  2.881778
22  2.854793
23  2.827789
24  2.800818
25  2.773808
26  2.746802
27  2.719831
28  2.692815
29  2.665826
..       ...
70  1.558965
71  1.531973
72  1.504979
73  1.477984
74  1.450977
75  1.423983
76  1.396982
77  1.369999
78  1.342975
79  1.315986
80  1.288991
81  1.261991
82  1.234996
83  1.207998
84  1.180992
85  1.154000
86  1.127017
87  1.099995
88  1.073015
89  1.046015
90  1.019032
91  0.992028
92  0.965025
93  0.938027
94  0.911038
95  0.884014
96  0.857028
97  0.830022
98  0.803039
99  0.776047

[100 rows x 1 columns]
